In [7]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [8]:
# Generate dataset
data, labels = make_moons(n_samples=100, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
y_train = 2 * y_train - 1  # Convert labels to {+1, -1}
y_test = 2 * y_test - 1

In [9]:
# Define quantum device
dev = qml.device("default.qubit", wires=2)

In [10]:
@qml.qnode(dev)
def quantum_circuit(x1, x2):
    qml.RY(x1, wires=0)
    qml.RY(x2, wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.probs(wires=[0, 1])

In [11]:

def quantum_kernel(x1, x2):
    return np.dot(quantum_circuit(*x1), quantum_circuit(*x2))

In [12]:
def compute_kernel_matrix(X1, X2):
    n1, n2 = len(X1), len(X2)
    kernel_matrix = np.zeros((n1, n2))
    for i in range(n1):
        for j in range(n2):
            kernel_matrix[i, j] = quantum_kernel(X1[i], X2[j])
    return kernel_matrix

In [13]:
# Compute kernel matrices
K_train = compute_kernel_matrix(X_train, X_train)
K_train

tensor([[0.64430957, 0.62124484, 0.76509357, ..., 0.26697223, 0.36107112,
         0.57325136],
        [0.62124484, 0.60606876, 0.72071387, ..., 0.23832972, 0.32920979,
         0.53422166],
        [0.76509357, 0.72071387, 0.95533466, ..., 0.3163554 , 0.43757679,
         0.71071887],
        ...,
        [0.26697223, 0.23832972, 0.3163554 , ..., 0.55097099, 0.50794771,
         0.41056151],
        [0.36107112, 0.32920979, 0.43757679, ..., 0.50794771, 0.495813  ,
         0.46810675],
        [0.57325136, 0.53422166, 0.71071887, ..., 0.41056151, 0.46810675,
         0.59758914]], requires_grad=True)

In [14]:
K_test = compute_kernel_matrix(X_test, X_train)
K_test

tensor([[0.67875019, 0.65053119, 0.81736395, ..., 0.27672641, 0.37896799,
         0.60946573],
        [0.63384734, 0.60224335, 0.77004721, ..., 0.32028098, 0.40561056,
         0.59789213],
        [0.48921619, 0.45561591, 0.59796698, ..., 0.42594612, 0.45929582,
         0.5342281 ],
        ...,
        [0.59344832, 0.55825414, 0.72810014, ..., 0.36819757, 0.43680951,
         0.59131407],
        [0.62783026, 0.58620194, 0.7824116 , ..., 0.39111943, 0.46583131,
         0.63402982],
        [0.76673618, 0.72088026, 0.96033676, ..., 0.32398565, 0.4447649 ,
         0.71689111]], requires_grad=True)

In [15]:
# Train SVM
svm = SVC(kernel="precomputed")
svm.fit(K_train, y_train)

SVC(kernel='precomputed')

In [16]:
# Predict and evaluate
y_pred = svm.predict(K_test)
accuracy = accuracy_score(y_test, y_pred)

In [17]:
print(f"Quantum SVM Accuracy: {accuracy:.2f}")

Quantum SVM Accuracy: 0.37
